# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,ust'-ilimsk,58.0006,102.6619,68.25,78,67,2.59,RU,1691542570
1,1,port-aux-francais,-49.3500,70.2167,34.83,71,60,17.72,TF,1691542570
2,2,thompson,55.7435,-97.8558,55.56,88,100,12.66,CA,1691542571
3,3,hamilton,39.1834,-84.5333,78.24,68,20,10.36,US,1691542475
4,4,dudinka,69.4058,86.1778,65.62,57,94,1.66,RU,1691542571


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width=700,
    frame_height=500,

    size = "Humidity",
    color = "City"
    )

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_new = city_data_df.loc[(city_data_df["Max Temp"] < 90) & (city_data_df["Max Temp"] > 65) & (city_data_df["Cloudiness"] < 20) & (city_data_df["Wind Speed"] < 2)]

# Drop any rows with null values
city_data_new.dropna()

# Display sample data
city_data_new

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
115,115,lata,40.1629,-8.3327,69.93,86,0,1.59,PT,1691542601
139,139,barabai,-2.5833,115.3833,82.72,59,14,0.76,ID,1691542608
185,185,irtyshsk,53.3333,75.4572,69.78,51,7,0.87,KZ,1691542620
200,200,remire-montjoly,4.9167,-52.2667,80.64,83,0,1.14,GF,1691542624
222,222,cordele,31.9635,-83.7824,74.44,94,0,0.00,US,1691542630
293,293,carlisle,40.2015,-77.1889,78.84,53,4,1.01,US,1691542651
331,331,grinnell,41.7430,-92.7224,78.66,75,0,0.00,US,1691542660
338,338,la mira tumbiscatio,18.6833,-102.2833,84.65,65,2,1.99,MX,1691542663
500,500,shahritus,37.2637,68.1385,79.21,29,0,1.57,TJ,1691542708
505,505,matoury,4.8483,-52.3317,80.56,83,0,1.14,GF,1691542709


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_new[["City", "Country", "Lat", "Lng", "Humidity"]].copy()


# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
115,lata,PT,40.1629,-8.3327,86,
139,barabai,ID,-2.5833,115.3833,59,
185,irtyshsk,KZ,53.3333,75.4572,51,
200,remire-montjoly,GF,4.9167,-52.2667,83,
222,cordele,US,31.9635,-83.7824,94,
293,carlisle,US,40.2015,-77.1889,53,
331,grinnell,US,41.7430,-92.7224,75,
338,la mira tumbiscatio,MX,18.6833,-102.2833,65,
500,shahritus,TJ,37.2637,68.1385,29,
505,matoury,GF,4.8483,-52.3317,83,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "apiKey": geoapify_key,
    "categories": "accommodation.hotel"
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lon = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
lata - nearest hotel: Residencial Botânico
barabai - nearest hotel: Hotel Bouraq
irtyshsk - nearest hotel: No hotel found
remire-montjoly - nearest hotel: Complexe Belova
cordele - nearest hotel: Baymont by Wyndham Cordele
carlisle - nearest hotel: No hotel found
grinnell - nearest hotel: Comfort Inn & Suites
la mira tumbiscatio - nearest hotel: No hotel found
shahritus - nearest hotel: Marhabo Hotel
matoury - nearest hotel: La Marmotte


,City,Country,Lat,Lng,Humidity,Hotel Name
115,lata,PT,40.1629,-8.3327,86,Residencial Botânico
139,barabai,ID,-2.5833,115.3833,59,Hotel Bouraq
185,irtyshsk,KZ,53.3333,75.4572,51,No hotel found
200,remire-montjoly,GF,4.9167,-52.2667,83,Complexe Belova
222,cordele,US,31.9635,-83.7824,94,Baymont by Wyndham Cordele
293,carlisle,US,40.2015,-77.1889,53,No hotel found
331,grinnell,US,41.7430,-92.7224,75,Comfort Inn & Suites
338,la mira tumbiscatio,MX,18.6833,-102.2833,65,No hotel found
500,shahritus,TJ,37.2637,68.1385,29,Marhabo Hotel
505,matoury,GF,4.8483,-52.3317,83,La Marmotte


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width=700,
    frame_height=500,
    hover_cols = ["Hotel Name", "Country"],
    size = "Humidity",
    color = "City"
    )

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)